# Ottenimento prossimi match
Lo script deve permettere di prendere i prossimi 10 match. 
Verrà fatta una predizione su questi match, prima deve essere calcolata la media di ciascun match che è data dai 5 match precedenti.

In [116]:
from analysis import MatchAnalysis
import pandas as pd
import util_strings as utils
from ranking import Ranking
from football_predictions import FootballPredictions

In [117]:
championship = pd.read_csv(utils.championship_actual_season, index_col=0)
championship['date'] = pd.to_datetime(championship['date'], format='%Y-%m-%d')

championship.result = championship.result.astype(str)
new_matches = championship[championship.result == 'nan']

In [118]:
matches = new_matches[:10][['team1', 'team2', 'date', 'result']]
matches.rename(columns= {'team1': 'home', 'team2': 'away'}, inplace=True)

## Ottenimento testo FootballPrediction
Scaricamento del testo di ciascuna partita

In [119]:
fp = FootballPredictions(matches=matches)
fp.update_originalNotation()
fp.get_urls()
fp.get_predictions(utils.next_matches)
fp.recovery_games()
fp.matches_not_found()
fp.fix_dates(utils.next_matches)

100%|██████████| 4/4 [00:01<00:00,  2.07it/s]


In [120]:
from my_tokenizer import MyTokenizer

mt = MyTokenizer(fp.df)
#mt.feature_normalization()
mt.clean_text()
mt.open_vectorizer(utils.vectorizer)

In [121]:
tok = mt.vectorizer.transform(mt.cleaned_corpus)

In [122]:
import pickle

dt = pickle.load(open('ML model/ta decision tree 0.942 (no syn).model', 'rb'))
res = dt.predict(tok)
res

ValueError: X has 17615 features, but DecisionTreeClassifier is expecting 17833 features as input.

In [ ]:
cont = 0
for i, el in fp.df.iterrows():
    if (el.prediction == res[i]):
        cont += 1
          
print(cont)

2


Per ciascun match devo calcolare le statistiche date dalle medie dei 5 match precedenti e poi devo ottenere il testo da football_prediction

In [ ]:
"""r = Ranking("Serie A", '2022-2023')
r.read_matches(seasons=5)
ma = MatchAnalysis("files/Serie A/3 seasons.csv")
ma.set_ranking(r)
ma.read_diff_dataset()

ma.matches_fe_team['date'] = pd.to_datetime(ma.matches_fe_team['date'], format='%Y-%m-%d')
ma.create_team_dataset()

float_features_and_avg = [x for x in ma.matches_by_team[0].matches.columns if x != 'team1' and x != 'team2' and x != 'date' and x != 'result' and x != 'rank_h' and x != 'rank_a' and x != 'matchday' and x != 'stadium' and x != 'completed_passings' and x != 'total_passings' and x != 'corners']

for i, match in matches.iterrows(): 
    avgH, c = ma.get_team_by_name(match.team1).get_avg_last_X_matches(5, match.date, float_features_and_avg)
    avgA, c = ma.get_team_by_name(match.team2).get_avg_last_X_matches(5, match.date, float_features_and_avg)

    for col in float_features_and_avg: 
        diff = avgH[col] - avgA[col]
        matches.at[i, col] = diff

matches
"""

'r = Ranking("Serie A", \'2022-2023\')\nr.read_matches(seasons=5)\nma = MatchAnalysis("files/Serie A/3 seasons.csv")\nma.set_ranking(r)\nma.read_diff_dataset()\n\nma.matches_fe_team[\'date\'] = pd.to_datetime(ma.matches_fe_team[\'date\'], format=\'%Y-%m-%d\')\nma.create_team_dataset()\n\nfloat_features_and_avg = [x for x in ma.matches_by_team[0].matches.columns if x != \'team1\' and x != \'team2\' and x != \'date\' and x != \'result\' and x != \'rank_h\' and x != \'rank_a\' and x != \'matchday\' and x != \'stadium\' and x != \'completed_passings\' and x != \'total_passings\' and x != \'corners\']\n\nfor i, match in matches.iterrows(): \n    avgH, c = ma.get_team_by_name(match.team1).get_avg_last_X_matches(5, match.date, float_features_and_avg)\n    avgA, c = ma.get_team_by_name(match.team2).get_avg_last_X_matches(5, match.date, float_features_and_avg)\n\n    for col in float_features_and_avg: \n        diff = avgH[col] - avgA[col]\n        matches.at[i, col] = diff\n\nmatches\n'